In [1]:
import os
import OpenDartReader
import FinanceDataReader as fdr

In [3]:
my_api = "4cf147f9e5c9916736cd84370f3f51b0a4c3d94f"
dart = OpenDartReader(my_api)

stock_list = fdr.StockListing("KOSDAQ").dropna()

In [11]:
import numpy as np
import pandas as pd
def find_financial_indicators(stock_name, year, indicators):
    report = dart.finstate(stock_name, year) # 데이터 가져오기
    if report is None: # 리포트가 없다면 (참고: 리포트가 없으면 None을 반환함)
        # 리포트가 없으면 당기, 전기, 전전기 값 모두 제거
        data = [[stock_name, year] + [np.nan] * len(indicators)]
        data = [[stock_name, year-1] + [np.nan] * len(indicators)]
        data = [[stock_name, year-2] + [np.nan] * len(indicators)]
        return pd.DataFrame(data, columns = ["기업", "연도"] + indicators)
    
    else:
        report = report[report['account_nm'].isin(indicators)] # 관련 지표로 필터링
        if sum(report['fs_nm'] == "연결재무제표") > 0:
            # 연결재무제표 데이터가 있으면 연결재무제표를 사용
            report = report.loc[report['fs_nm'] == "연결재무제표"]
            
        else:
            # 연결재무제표 데이터가 없으면 일반재무제표를 사용
            report = report.loc[report['fs_nm'] == "재무제표"]
        
        data = []
        for y, c in zip([year, year-1, year-2], ['thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']):
            record = [stock_name, y]
            for indic in indicators:
                # account_nm이 indic인 행의 c 컬럼 값을 가져옴
                if sum(report['account_nm'] == indic) > 0: # 해당 지표가 있다면 추가 (지표가 없는 경우도 있음)
                    value = report.loc[report['account_nm'] == indic, c].iloc[0]
                else:
                    value = np.nan
                
                record.append(value)
            
            data.append(record)
        return pd.DataFrame(data, columns = ["기업", "연도"] + indicators)

In [12]:
import time 
indicators = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '당기순이익']
data = pd.DataFrame() # 이 데이터프레임에 각각의 데이터를 추가할 예정

for idx, stock_name in enumerate(stock_list):
    print(idx+1, "/", len(stock_list)) # 현재까지 진행된 상황 출력
    for year in [2019, 2020, 2021]:
        try: # 재무제표 데이터가 잘 불러와지지 않는 경우가 있어, try - except으로 넘김
            result = find_financial_indicators(stock_name, year, indicators) # 재무지표 데이터 
        except:
            pass  
        data = pd.concat([data, result], axis = 0, ignore_index = True) # data에 부착
        time.sleep(0.5)

1 / 1431


TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid